# Import Packages

In [1]:
%matplotlib inline
%env KERAS_BACKEND=tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dense
from keras.datasets import cifar100
from keras.utils import np_utils
from keras.preprocessing import image
from spp.SpatialPyramidPooling import SpatialPyramidPooling    # 需引入相關檔案 https://github.com/yhenon/keras-spp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import os 
from PIL import Image
from io import BytesIO

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


# 引入CIFAR約10000張圖片作為非meme的圖片

In [3]:
(CIFAR_train,  _), _ = cifar100.load_data(label_mode='fine')    # 引入CIFAR，實際上只需要圖片的部分
# 需要將圖檔放大為128*128以利訓練
imgs = []
sele_img = CIFAR_train[:8000]    # 只需要八千筆就好
for k in range(8000):
    pre_img = sele_img[k]/255
    span_img = np.zeros((128,128, 3))
    for u in range(32):
        for v in range(32):
            for i in range(4):
                for j in range(4):
                    span_img[4*u+i][4*v+j] = pre_img[u, v, :3]    # 將圖片放大成128*128
#     span_img = np.expand_dims(span_img, axis=0)    # 增加第一個batch維度
    imgs.append(span_img)    # 把圖片數組加到一個列表裡面
    imgs1 = imgs
del sele_img

# 從硬碟讀入meme與cifar100打散，並做成gnerator

In [7]:
import numpy as np
import imageio
TOTAL_IMG_NUM = 8000 + 8000
BATCH_SIZE = 256
def _generator(batch_size = BATCH_SIZE):# 用來fit_generator
    while True:
        rd = np.random.permutation(TOTAL_IMG_NUM)
        for i in range(TOTAL_IMG_NUM // batch_size):
            imgs = []
            result = []
            for j in rd[i*batch_size:(i+1)*batch_size]:
                if j >= 8000:
                    imgs.append(imgs1[j - 8000])
                    result.append(0)
                else:
                    imgs.append(imageio.imread("./img/meme" + str(j) + ".png")) # (路徑改成自己的)
                    result.append(1)
            yield np.array(imgs), np.ones((batch_size))
        imgs = []
        result = []
        for j in rd[(i+1)*batch_size:]:
            if j >= 8000:
                imgs.append(imgs1[j - 8000])
                result.append(0)
            else:
                imgs.append(imageio.imread("./img/meme" + str(j) + ".png")) # (路徑改成自己的)
                result.append(1)
        yield np.array(imgs), np.array(result)

# 資料處理完畢，開始架設神經網路

In [5]:
num_channels = 3
num_classes = 2

model = Sequential()
model.add(Conv2D(2, (9,9), padding='same', input_shape=(None, None, 3)))   #圖片大小不固定所以用NONE
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   #第一層
model.add(Conv2D(4, (8,8), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   #第二層
model.add(Conv2D(8, (7,7), padding='same'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))   #第三層(怕最後矩陣變太小這裡先不做pooling)
model.add(Conv2D(16, (6,6), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   #第四層
model.add(Conv2D(32, (5,5), padding='same'))
model.add(Activation('relu'))
#這裡運用SPP讓結果的大小是固定的
model.add(SpatialPyramidPooling([1, 2, 4])) #會輸出(1+4+16)=21的大小
model.add(Dense(1))   #搭配上行是一個21*1的FC layer
model.add(Activation('sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 2)     488       
_________________________________________________________________
activation_1 (Activation)    (None, None, None, 2)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 2)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 4)     516       
_________________________________________________________________
activation_2 (Activation)    (None, None, None, 4)     0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, None, None, 4)     0         
_________________________________________________________________
conv

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    #learning rate不敢亂動QQ

model.fit_generator(_generator(), steps_per_epoch = TOTAL_IMG_NUM // BATCH_SIZE + 1, epochs=10)

Epoch 1/10
70/70 [==============================] - 392s 6s/step - loss: 0.1060 - acc: 0.9934
Epoch 2/10
70/70 [==============================] - 391s 6s/step - loss: 0.0870 - acc: 0.9945
Epoch 3/10
70/70 [==============================] - 475s 7s/step - loss: 0.1223 - acc: 0.9923
Epoch 4/10
70/70 [==============================] - 556s 8s/step - loss: 0.0897 - acc: 0.9944
Epoch 5/10
70/70 [==============================] - 455s 6s/step - loss: 0.1033 - acc: 0.9935
Epoch 6/10
61/70 [=========================>....] - ETA: 52s - loss: 1.1921e-07 - acc: 1.0000

# 把model存起來

In [ ]:
model.save('finalprojectmodel.h5')

# 測試一下train好的model

In [ ]:
x_test0 = CIFAR_train[8000:9747] 
y_test0 = np.zeros(1747)
x_test1 = np.array([imageio.imread("./img/meme" + str(j) + ".png") for j in range(8000, 9747)])
y_test1 = np.ones(1747)
x_test = np.concate(x_test0, x_test1)
y_test = np.concate(y_test0, y_test1)
score = model.evaluate(x_test, y_test)
print(f'測試資料的 loss: {score[0]:.5f}')
print(f'測試資料的正確率: {score[1]}')